In [46]:
import random
import pandas as pd
import numpy as np
import psutil
import time
import os

In [47]:
room_excel = pd.read_excel("Dataset/Gedung TULT.xlsx", usecols=['Nama Ruangan'])
room_list = room_excel['Nama Ruangan'].tolist()
room_list

['TULT-0601',
 'TULT-0602',
 'TULT-0603',
 'TULT-0701',
 'TULT-0702',
 'TULT-0703',
 'TULT-0706',
 'TULT-0707',
 'TULT-0709',
 'TULT-0710',
 'TULT-0711',
 'TULT-0714',
 'TULT-0715',
 'TULT-0716',
 'TULT-1507',
 'TULT-1509',
 'TULT-1510']

In [48]:
class_excel = pd.read_excel("Dataset/Kelas FIF.xlsx", sheet_name="Kelas FIF", usecols=['Kelas FIF'])
class_list = class_excel['Kelas FIF'].tolist()
class_list

['IF-47-01',
 'IF-47-02',
 'IF-47-03',
 'IF-47-04',
 'IF-47-05',
 'IF-47-06',
 'IF-47-07',
 'IF-47-08',
 'IF-47-09',
 'IF-47-10',
 'IF-47-11',
 'IF-47-12',
 'IF-47-INT',
 'IT-47-01',
 'IT-47-02',
 'IT-47-03',
 'IT-47-04',
 'SE-47-01',
 'SE-47-02',
 'SE-47-03',
 'SE-47-04',
 'DS-47-01',
 'DS-47-02',
 'DS-47-03']

In [49]:
subject_excel = pd.read_excel("Dataset/Mata Kuliah FIF.xlsx", sheet_name="FIF Semester 1", usecols=['Kode'])
subject_list = subject_excel['Kode'].tolist()
subject_list

['UKJXB2',
 'UAJXA2',
 'CII1E3',
 'CII1A3',
 'CII1C2',
 'CII1B3',
 'CII1D3 ',
 'CTJ1A2',
 'CRI1A2',
 'CRI1B3',
 'CSI1A2']

In [50]:
# Read the dataset and filter by status
dosen_dataset = pd.read_excel("Dataset/data dosen.xlsx", sheet_name="all dosen", usecols=['Kode', 'Status'])

In [51]:
rooms = room_list
students_groups = class_list
subjects = subject_list
permanent_lectures = dosen_dataset.loc[dosen_dataset['Status'] == 1, 'Kode'].tolist()
non_permanent_lectures = dosen_dataset.loc[dosen_dataset['Status'] == 0, 'Kode'].tolist()
lecturers = permanent_lectures + non_permanent_lectures
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
timeslots = ['timeslot1', 'timeslot2', 'timeslot3', 'timeslot4', 'timeslot5', 'timeslot6', 'timeslot7', 'timeslot8', 'timeslot9', 'timeslot10', 'timeslot11', 'timeslot12']

In [52]:
# Course restrictions
course_restrictions = {
    'UAJXA2': ['IF'],
    'CTJ1A2': ['IT'],
    'CRI1A2': ['SE'],
    'CRI1B3': ['SE'],
    'CSI1A2': ['DS']
}

In [53]:
# Hyperparameters
num_ants = 15
num_iterations = 150
decay = 0.5
alpha = 1
beta = 2

In [54]:
# Monitor resource usage
start_time = time.time()

cpu_usage = []
memory_usage_before = []
memory_usage_after = []
disk_usage_before = []
disk_usage_after = []

def monitor_resource_usage_before():
    cpu_usage.append(psutil.cpu_percent(interval=None))
    memory_usage_before.append(psutil.virtual_memory().used / (1024 ** 2))  # in MB
    disk_usage_before.append(psutil.disk_usage('/').used / (1024 ** 2))  # in MB

def monitor_resource_usage_after():
    cpu_usage.append(psutil.cpu_percent(interval=None))
    memory_usage_after.append(psutil.virtual_memory().used / (1024 ** 2))  # in MB
    disk_usage_after.append(psutil.disk_usage('/').used / (1024 ** 2))  # in MB

In [55]:
# Record resource usage before solving
monitor_resource_usage_before()

In [56]:
# Timeslots to minimize usage
undesirable_timeslots = ['timeslot1', 'timeslot5', 'timeslot6', 'timeslot11']

# Initial pheromone levels
# pheromone = np.ones((len(subjects), len(lecturers), len(students_groups), len(rooms), len(timeslots)))
pheromone = [[[1 for _ in timeslots] for _ in rooms] for _ in days]
pheromone_lecturer = {lecturer: 1 for lecturer in lecturers}

# Define the fitness function
def fitness(schedule):
    penalty = 0
    room_timeslot_usage = {}
    lecturer_course_group = {}
    timeslot_count = {room: {day: 0 for day in days} for room in rooms}
    group_timeslot_count = {group: {day: 0 for day in days} for group in students_groups}

    room_conflicts = 0
    lecturer_conflicts = 0
    undesirable_timeslot_penalty = 0
    empty_timeslot_penalty = 0
    specific_rules_penalty = 0

    for group in schedule:
        for (course, day, room, timeslot, lecturer) in schedule[group]:
            # Check room usage
            if (day, room, timeslot) in room_timeslot_usage:
                room_conflicts += 1
            else:
                room_timeslot_usage[(day, room, timeslot)] = group
                timeslot_count[room][day] += 1
                group_timeslot_count[group][day] += 1

            # Check lecturer constraints
            if (lecturer, course, group) in lecturer_course_group:
                lecturer_conflicts += 1
            else:
                lecturer_course_group[(lecturer, course, group)] = True

            # Additional penalty for undesirable timeslots
            if timeslot in undesirable_timeslots:
                undesirable_timeslot_penalty += 0.5

            # Specific rule: No permanent lecturers on timeslot4, timeslot5, timeslot6 on Thursdays
            if day == 'Thursday' and timeslot in ['timeslot4', 'timeslot5', 'timeslot6'] and lecturer in permanent_lectures:
                specific_rules_penalty += 1

            # Specific rule: No courses in timeslot5 and timeslot6 on Friday
            if day == 'Friday' and timeslot in ['timeslot5', 'timeslot6']:
                specific_rules_penalty += 1

            # Specific rule: Rooms TULT-1507, TULT-1509, TULT-1510 only for IF-47-INT
            if room in ['TULT-1507', 'TULT-1509', 'TULT-1510'] and group != 'IF-47-INT':
                specific_rules_penalty += 1

            # One course per timeslot per day per group
            if group_timeslot_count[group][day] > 1:
                specific_rules_penalty += 1

    # Penalty for empty timeslots and unbalanced room usage
    for room in rooms:
        for day in days:
            empty_timeslot_penalty += (len(timeslots) - timeslot_count[room][day]) * 0.1  # Less penalty for fewer empty slots

    total_penalty = (room_conflicts + lecturer_conflicts + undesirable_timeslot_penalty + empty_timeslot_penalty + specific_rules_penalty)

    return total_penalty



In [57]:
# Check if a course is scheduled
def is_course_scheduled(schedule, day, room, timeslot):
    for group in schedule:
        for (course, d, r, t, lecturer) in schedule[group]:
            if d == days[day] and r == rooms[room] and t == timeslots[timeslot]:
                return True
    return False

# Check if a lecturer is scheduled
def is_lecturer_scheduled(schedule, lecturer):
    for group in schedule:
        for (course, day, room, timeslot, l) in schedule[group]:
            if l == lecturer:
                return True
    return False

In [58]:
# Create the initial population
def create_schedule():
    schedule = {group: [] for group in students_groups}
    if_47_int_rooms = ['TULT-1507', 'TULT-1509', 'TULT-1510']

    for group in students_groups:
        for course in subjects:
            # Ensure the course is allowed for the group
            if course in course_restrictions:
                if not any(group.startswith(prefix) for prefix in course_restrictions[course]):
                    continue  # Skip if the course is restricted and the group doesn't match

            try:
                course_length = int(course[-1])  # Get the length of the course from the last digit
            except ValueError:
                continue  # Skip if the course code does not end with a digit

            while True:
                day = random.choice(days)
                # Ensure IF-47-INT uses the specific rooms
                if group == 'IF-47-INT':
                    room = random.choice(if_47_int_rooms)
                else:
                    room = random.choice([r for r in rooms if r not in if_47_int_rooms])
                
                start_timeslot_index = random.randint(0, len(timeslots) - course_length)
                timeslot_range = timeslots[start_timeslot_index:start_timeslot_index + course_length]
                lecturer = random.choice(lecturers)

                # Check if any timeslot in the range is already taken for the selected day and room
                if any([(day, room, timeslot) in [(d, r, t) for c, d, r, t, l in schedule[group]] for timeslot in timeslot_range]):
                    continue  # Skip if any timeslot in the range is already taken

                # Check room restriction for specific groups
                if group != 'IF-47-INT' and room in if_47_int_rooms:
                    continue  # Skip if room restriction is violated

                # Check permanent lecturer restriction on Thursdays
                if day == 'Thursday' and any(timeslot in ['timeslot4', 'timeslot5', 'timeslot6'] for timeslot in timeslot_range) and lecturer in permanent_lectures:
                    continue  # Skip if permanent lecturer rule is violated

                # Check Friday timeslot restriction
                if day == 'Friday' and any(timeslot in ['timeslot5', 'timeslot6'] for timeslot in timeslot_range):
                    continue  # Skip if Friday timeslot rule is violated

                # If all checks pass, add the course to the schedule
                for timeslot in timeslot_range:
                    schedule[group].append((course, day, room, timeslot, lecturer))
                break  # Exit the while loop once a valid assignment is made

    return schedule



In [59]:
# Update pheromones
def update_pheromones(pheromone, pheromone_lecturer, all_schedules):
    for day in range(len(days)):
        for room in range(len(rooms)):
            for timeslot in range(len(timeslots)):
                pheromone[day][room][timeslot] *= decay
                for schedule in all_schedules:
                    if is_course_scheduled(schedule, day, room, timeslot):
                        pheromone[day][room][timeslot] += 1 / (1 + fitness(schedule))
    
    for lecturer in lecturers:
        pheromone_lecturer[lecturer] *= decay
        for schedule in all_schedules:
            if is_lecturer_scheduled(schedule, lecturer):
                pheromone_lecturer[lecturer] += 1 / (1 + fitness(schedule))

    return pheromone, pheromone_lecturer



# Main ACO loop
# def aco():
#     best_schedule = None
#     best_fitness = float('inf')
#     for iteration in range(num_iterations):
#         all_schedules = []
#         for ant in range(num_ants):
#             schedule = create_schedule()
#             all_schedules.append(schedule)
#             # check constraint
#             fit = fitness(schedule)
#             if fit < best_fitness:
#                 best_fitness = fit
#                 best_schedule = schedule
#         update_pheromones(pheromone, pheromone_lecturer, all_schedules)
#     return best_schedule, best_fitness



In [60]:
# Main ACO loop
def aco():
    best_schedule = None
    best_fitness = float('inf')
    for iteration in range(num_iterations):
        all_schedules = []
        for ant in range(num_ants):
            schedule = create_schedule()
            all_schedules.append(schedule)
            fit = fitness(schedule)
            if fit < best_fitness:
                best_fitness = fit
                best_schedule = schedule
        update_pheromones(pheromone, pheromone_lecturer, all_schedules)
    return best_schedule, best_fitness

In [61]:
end_time = time.time()

# Record resource usage after solving
monitor_resource_usage_after()

# Print resource usage statistics
print(f"Time taken to solve: {end_time - start_time:.2f} seconds")
print(f"CPU usage before: {cpu_usage[0]:.2f}%")
print(f"CPU usage after: {cpu_usage[1]:.2f}%")
print(f"Memory usage before: {memory_usage_before[0]:.2f} MB")
print(f"Memory usage after: {memory_usage_after[0]:.2f} MB")
print(f"Disk usage before: {disk_usage_before[0]:.2f} MB")
print(f"Disk usage after: {disk_usage_after[0]:.2f} MB")

Time taken to solve: 0.17 seconds
CPU usage before: 9.30%
CPU usage after: 10.40%
Memory usage before: 9355.02 MB
Memory usage after: 9355.94 MB
Disk usage before: 292575.03 MB
Disk usage after: 292575.03 MB


In [62]:
# Display the best schedule in a timetable form categorized by room
def print_timetable_by_room(schedule):
    timetable = {room: {day: {timeslot: None for timeslot in timeslots} for day in days} for room in rooms}
    for group in schedule:
        for (course, day, room, timeslot, lecturer) in schedule[group]:
            timetable[room][day][timeslot] = (course, group, lecturer)

    for room in rooms:
        print(f"Timetable for {room}:")
        for day in days:
            print(f"  {day}:")
            for timeslot in timeslots:
                entry = timetable[room][day][timeslot]
                if entry:
                    course, group, lecturer = entry
                    print(f"    {timeslot}: {course} for {group} with {lecturer}")
                
        print()

best_schedule, best_fitness = aco()
print_timetable_by_room(best_schedule)
print("Best Fitness:", best_fitness)

Timetable for TULT-0601:
  Monday:
    timeslot4: CII1B3 for IT-47-04 with ERW
    timeslot5: CII1B3 for IT-47-04 with ERW
    timeslot6: CII1B3 for IT-47-04 with ERW
    timeslot10: UKJXB2 for IF-47-10 with IDV
    timeslot11: UKJXB2 for IF-47-10 with IDV
  Tuesday:
  Wednesday:
  Thursday:
    timeslot1: CTJ1A2 for IT-47-02 with DTO
    timeslot2: CTJ1A2 for IT-47-02 with DTO
  Friday:
  Saturday:
    timeslot2: CII1C2 for IF-47-01 with DDR
    timeslot3: CII1C2 for IF-47-01 with DDR
    timeslot8: CII1C2 for IT-47-03 with IPL
    timeslot9: CII1C2 for IT-47-03 with IPL

Timetable for TULT-0602:
  Monday:
    timeslot9: CSI1A2 for DS-47-03 with ALH
    timeslot10: CSI1A2 for DS-47-03 with ALH
  Tuesday:
    timeslot5: CII1A3 for IF-47-07 with HMT
    timeslot6: CII1A3 for IF-47-07 with HMT
    timeslot7: UKJXB2 for SE-47-04 with FKI
    timeslot8: UKJXB2 for SE-47-04 with FKI
  Wednesday:
    timeslot7: CTJ1A2 for IT-47-03 with FMH
    timeslot8: CII1E3 for SE-47-02 with EDW
    time